In [93]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from citipy import citipy
import requests
import os
import gmaps
from config import weather_api_key
from config import g_key

In [94]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [95]:
#load weatherpy df
city_data_to_load = os.path.join("data","WeatherPy_database.csv")
city_data_df = pd.read_csv(city_data_to_load)
city_data_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Busselton,AU,-33.65,115.33,59.00,80,15,19.10,few clouds,0.00,0.0
1,1,Dingle,PH,11.00,122.67,79.14,84,100,4.63,moderate rain,1.12,0.0
2,2,East London,ZA,-33.02,27.91,73.40,28,0,5.82,clear sky,0.00,0.0
3,3,Blackwater,AU,-23.58,148.88,57.87,73,46,7.85,scattered clouds,0.00,0.0
4,4,Mataura,NZ,-46.19,168.86,39.00,83,99,3.00,overcast clouds,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
560,560,Gold Coast,AU,-28.00,153.43,61.00,77,90,8.05,overcast clouds,0.00,0.0
561,561,Varzea Da Palma,BR,-17.60,-44.73,75.13,49,0,4.68,clear sky,0.00,0.0
562,562,Kentau,KZ,43.52,68.51,72.05,20,0,18.01,clear sky,0.00,0.0
563,563,Bosanska Gradiska,BA,45.14,17.25,89.60,38,20,4.70,few clouds,0.00,0.0


In [96]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_question = str(input("Do you want it to be raining? (yes/no) "))
snow_question = str(input("Do you want it to be snowing? (yes/no) "))
# Filter the dataset to find the cities that fit the criteria.

if rain_question =="yes" and snow_question =="yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] > 0.0) &
                                           (city_data_df["Snow (inches)"] > 0.0)]
    
elif rain_question =="yes" and snow_question =="no":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] > 0.0) &
                                           (city_data_df["Snow (inches)"] == 0.0)]
    
elif rain_question =="no" and snow_question =="yes":
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                           (city_data_df["Max Temp"] >= min_temp) &
                                           (city_data_df["Rain (inches)"] == 0.0) &
                                           (city_data_df["Snow (inches)"] > 0.0)]    
    
else:
    preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                          (city_data_df["Max Temp"] >= min_temp) &
                                          (city_data_df["Rain (inches)"] == 0.0) &
                                          (city_data_df["Snow (inches)"] == 0.0)]
    
    
preferred_cities_df

What is the minimum temperature you would like for your trip? 76
What is the maximum temperature you would like for your trip? 95
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
17,17,Praia,CV,14.92,-23.51,84.20,58,40,17.22,scattered clouds,0.0,0.0
19,19,Jetpur,IN,21.73,70.62,92.30,49,100,14.38,overcast clouds,0.0,0.0
20,20,Kahului,US,20.89,-156.47,78.80,74,90,8.05,overcast clouds,0.0,0.0
34,34,Eskisehir,TR,39.78,30.52,80.60,47,20,5.82,few clouds,0.0,0.0
38,38,Faanui,PF,-16.48,-151.75,76.78,76,34,15.97,scattered clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
552,552,Trairi,BR,-3.28,-39.27,82.26,71,48,8.77,scattered clouds,0.0,0.0
553,553,Bambanglipuro,ID,-7.95,110.28,76.08,83,0,8.28,clear sky,0.0,0.0
558,558,Toul,FR,48.67,5.92,80.60,61,44,14.99,scattered clouds,0.0,0.0
563,563,Bosanska Gradiska,BA,45.14,17.25,89.60,38,20,4.70,few clouds,0.0,0.0


In [97]:
# Create DataFrame to store hotel names along with city, country, max temp, description, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
541,Tateyama,JP,79.00,overcast clouds,34.98,139.87,
543,Baykit,RU,76.64,overcast clouds,61.67,96.37,
546,Tiarei,PF,77.00,broken clouds,-17.53,-149.33,
548,Lazaro Cardenas,MX,77.74,overcast clouds,17.96,-102.20,
551,Los Llanos De Aridane,ES,77.00,broken clouds,28.66,-17.92,
552,Trairi,BR,82.26,scattered clouds,-3.28,-39.27,
553,Bambanglipuro,ID,76.08,clear sky,-7.95,110.28,
558,Toul,FR,80.60,scattered clouds,48.67,5.92,
563,Bosanska Gradiska,BA,89.60,few clouds,45.14,17.25,
564,Puerto Escondido,MX,77.00,broken clouds,15.85,-97.07,


In [98]:
hotel_df.count()

City                   187
Country                187
Max Temp               187
Current Description    187
Lat                    187
Lng                    187
Hotel Name             187
dtype: int64

In [99]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError, KeyError):
        print("No hotel")
    else:
        if hotel_df["Hotel Name"] is None:
            pass

No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel
No hotel


In [100]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Praia,CV,84.20,scattered clouds,14.92,-23.51,LT Aparthotel
19,Jetpur,IN,92.30,overcast clouds,21.73,70.62,Hotel The Grand Bansari
20,Kahului,US,78.80,overcast clouds,20.89,-156.47,Maui Seaside Hotel
34,Eskisehir,TR,80.60,few clouds,39.78,30.52,Hotel ibis Eskisehir
38,Faanui,PF,76.78,scattered clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
43,Hithadhoo,MV,84.76,overcast clouds,-0.60,73.08,Scoop Guest House
44,Upata,VE,78.01,broken clouds,8.01,-62.40,Posada Villa Nela
45,Kapaa,US,78.80,overcast clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
46,Labuhan,ID,79.39,clear sky,-6.88,112.21,bador sepret
48,Dong Hoi,VN,84.09,overcast clouds,17.48,106.60,Nam Long Hotel


In [101]:
# Remove rows with blank Hotel(nothing in 5km)
empty = hotel_df["Hotel Name"] != ""
hotel_df = hotel_df[empty]
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
17,Praia,CV,84.20,scattered clouds,14.92,-23.51,LT Aparthotel
19,Jetpur,IN,92.30,overcast clouds,21.73,70.62,Hotel The Grand Bansari
20,Kahului,US,78.80,overcast clouds,20.89,-156.47,Maui Seaside Hotel
34,Eskisehir,TR,80.60,few clouds,39.78,30.52,Hotel ibis Eskisehir
38,Faanui,PF,76.78,scattered clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
...,...,...,...,...,...,...,...
552,Trairi,BR,82.26,scattered clouds,-3.28,-39.27,Pousada Field Lilies - Trairi
553,Bambanglipuro,ID,76.08,clear sky,-7.95,110.28,Puri Brata
558,Toul,FR,80.60,scattered clouds,48.67,5.92,Abc Hotel
563,Bosanska Gradiska,BA,89.60,few clouds,45.14,17.25,Hostel Art Gradiska


In [102]:
# Create the output file (CSV).
output_data_file2 = "data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file2, index_label="City_ID")

In [103]:
#create popup box
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F </dd>
</dl>
"""

In [104]:
# Store the DF
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [105]:
# Add a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))